# Base parameters and combinations with QR

In the following, the QR decomposition is used to find columns in a matrix (e.g. dynamics regressor) that are independent and therefore are a choice of a basis for the identifiable subspace. It is then determined which linear combinations of the other dependent columns are forming the base columns/parameters in each of those. This is also necessary for the std essential parameters.

In [3]:
import numpy as np; np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

In [38]:
a = np.array([[1.0, 2.0,  .0,  .0, 2.0, 3.0],
              [1.0, 2.0,  .0,  .0, 1.0, 3.0],
              [ .0,  .0, 1.0, 5.0,  .0, .0],
              [ .0,  .0, 1.0, 5.0,  .0, .0],
              [2.0, 4.0,  .0,  .0,  .0, 6.0],
              [ .0,  .0, 0.5, 2.5,  .0, .0]])
print(a.shape)
r = np.linalg.matrix_rank(a)
r

(6, 6)


3

Test Matrix $a$ is rank deficient (rank $r<m$). The rank 3 means there are 3 sets of dependent columns, each with 1 or more columns in it. The columns $0,1$ and $2,3$ are linearly dependent. Column 4 is linearly independent from all other columns. Finding independent columns will give a single representant of each set, while the choice is non-unique if any set has more than one element. If a column is independent, there can be other columns that are dependent on it but it can also be a single element in its set.

$$a\cdot x = t$$

In [40]:
x = np.array([1.0,2.0,0.838,4.192,4.5, 2.3])  #param
t = a.dot(x)                         #result
t

array([ 20.900,  16.400,  21.798,  21.798,  23.800,  10.899])

## Sets of dependent columns

Assuming $m \times n$ Matrix $a$ with $m>=n$.

In [45]:
import scipy.linalg as sla
Qp,Rp,P = sla.qr(a, pivoting=True, check_finite=True)
print(Rp.diagonal())
print(P)
print()
#Q,R = np.linalg.qr(a, mode='complete')
#print(R.diagonal())

[-7.500 -7.348  1.871 -0.000  0.000 -0.000]
[3 5 4 1 0 2]



Find selection of columns that are linearly independent to each other. For non-pivoting QR, the columns in R with diagonal elements that are not zero are independent. For pivoting QR this does not work. However, as there are as many independent columns as the rank of the matrix, taking the first rank columns of the permutation also gives the independent column indices but in different order. The choices are not the same but from the same sets of dependent columns.

In [47]:
tol = 1e-6
#print(R.diagonal())
#indepsQR = np.where(np.abs(R.diagonal()) > tol)[0]
#print(indepsQR)

print("rank: {}".format(np.where(np.abs(Rp.diagonal()) > tol)[0].size ))   #rank with pivoting QR
indepsQRP = P[0:r]
print(indepsQRP)


rank: 3
[3 5 4]


In [48]:
#create proper permutation matrix
Pp = np.zeros((P.size, P.size))
for i in P:
    Pp[i,P[i]]=1

In [49]:
#Gautier 1990, 3-4: linear relationships
W1W2 = a.dot(Pp)
W1 = W1W2[:,0:indepsQRP.size]
W2 = W1W2[:,indepsQRP.size:a.shape[1]]
#Q,R = np.linalg.qr(W1W2)
#print(Q)
#print(R.diagonal())
#corresponds to QR with pivoting
#print(Qp)
#print(Rp.diagonal())

#dependent columns W1 as linear combinations of independent columns W1
R1 = Rp[0:indepsQRP.size,0:indepsQRP.size]
R2 = Rp[0:indepsQRP.size,indepsQRP.size:]
linDeps = np.linalg.inv(R1).dot(R2)
print(linDeps)
print(indepsQRP)
#should be equal
#print(W2)
#print(W1.dot(linDeps))

[[ 0.000  0.000  0.200]
 [ 0.667  0.333  0.000]
 [ 0.000  0.000 -0.000]]
[3 5 4]


In [59]:
print("P:", P)
print("a:\n", a)
for i in range(0,linDeps.shape[0]):
    for j in range(0,linDeps.shape[1]):
        if np.abs(linDeps[i,j]) > 0.1:
            dep_org_col = P[indepsQRP.size+j]
            indep_org_col = P[i]

            print(
                '''col {} in W2(col {} in a) is a linear combination of col {} in W1 (col {} in a) with factor {}'''\
                   .format(i, dep_org_col, j, indep_org_col, linDeps[i,j]))
a[:, 5]*2/3 - a[:, 1]

P: [3 5 4 1 0 2]
a:
 [[ 1.000  2.000  0.000  0.000  2.000  3.000]
 [ 1.000  2.000  0.000  0.000  1.000  3.000]
 [ 0.000  0.000  1.000  5.000  0.000  0.000]
 [ 0.000  0.000  1.000  5.000  0.000  0.000]
 [ 2.000  4.000  0.000  0.000  0.000  6.000]
 [ 0.000  0.000  0.500  2.500  0.000  0.000]]
col 0 in W2(col 2 in a) is a linear combination of col 2 in W1 (col 3 in a) with factor 0.2
col 1 in W2(col 1 in a) is a linear combination of col 0 in W1 (col 5 in a) with factor 0.6666666666666667
col 1 in W2(col 0 in a) is a linear combination of col 1 in W1 (col 5 in a) with factor 0.33333333333333337


array([ 0.000,  0.000,  0.000,  0.000,  0.000,  0.000])

## Getting the base projection matrix

The first $r$ orthogonal columns of $Q$ form a basis $B$ of the column space of $A$ in case a is a square matrix. In the rectangular case, the QR decomposition of $A^T$ seems to be possible but maybe not always solvable?. 
$B$ can be used to project the matrix $A$ with regard to a vector $v$ to a matrix $A_\text{base}$ with regard to a vector $v_\text{base}$.

In [60]:
Qpt,Rpt,Pt = sla.qr(a.T, pivoting=True, mode='economic')
print(Qpt)
B = -Qpt[:, 0:r]
print(B)
#B = sla.orth(a)[0:a.shape[1], :]
#print(B)

[[-0.267  0.000 -0.000 -0.535 -0.799 -0.067]
 [-0.535  0.000 -0.000  0.775 -0.337 -0.028]
 [-0.000 -0.196  0.000  0.000  0.081 -0.977]
 [-0.000 -0.981 -0.000  0.000 -0.016  0.195]
 [-0.000 -0.000 -1.000  0.000  0.000  0.000]
 [-0.802 -0.000  0.000 -0.338  0.491  0.041]]
[[ 0.267 -0.000  0.000]
 [ 0.535 -0.000  0.000]
 [ 0.000  0.196 -0.000]
 [ 0.000  0.981  0.000]
 [ 0.000  0.000  1.000]
 [ 0.802  0.000 -0.000]]


$Pp^T XB = [XB1 ; XB2] = [X1 + X1^{-1}R2 X2;0]$

In [61]:
print(P)
Px = Pp.dot(x)
print('x:\t', x)
x1 = Px[0:r]
x2 = Px[r:]
print ('x1, x2:\t', x1, x2)
xb1 = x1 + np.linalg.inv(R1).dot(R2.dot(x2))
print (xb1)

[3 5 4 1 0 2]
x:	 [ 1.000  2.000  0.838  4.192  4.500  2.300]
x1, x2:	 [ 4.192  2.300  4.500] [ 2.000  1.000  0.838]
[ 4.360  3.967  4.500]


In [299]:
a_base = a.dot(B)
x_base = B.T.dot(x)
print('x_base:\t', x_base)
print("deps:")
for i in range(0, len(x_base)):
    print("{}, ".format(indepsQRP[i], deps[i]))
t_base = a_base.dot(x_base)
print('t:\t', t)
print('t_base:\t', t_base)
print('x:\t', x)
print ('x*:\t', B.dot(x_base))    #should give same x as in standard space if x was fully in base space already

# test nullspace
print(a.dot(np.eye(5) - B.dot(B.T)))

x_base:	 [ 4.275  2.236  4.500]
deps:
3, 
1, 
4, 
t:	 [ 14.000  9.500  21.798  21.798  10.000  10.899]
t_base:	 [ 14.000  9.500  21.798  21.798  10.000  10.899]
x:	 [ 1.000  2.000  0.838  4.192  4.500]
x*:	 [ 1.000  2.000  0.838  4.192  4.500]
[[-0.000  0.000 -0.000 -0.000  0.000]
 [-0.000  0.000 -0.000 -0.000  0.000]
 [-0.000 -0.000 -0.000 -0.000  0.000]
 [-0.000 -0.000 -0.000 -0.000  0.000]
 [ 0.000  0.000 -0.000 -0.000  0.000]
 [-0.000 -0.000 -0.000 -0.000  0.000]]


In [62]:
deps = np.linalg.inv(Rpt[0:r, 0:r]).dot(Rpt[0:r, r:])
print(deps)

[[ 0.000  0.250  0.000]
 [ 1.000  0.000  0.500]
 [-0.000  0.500 -0.000]]


### Base Projection with SVD

In [63]:
U, s, Vh = sla.svd(a, full_matrices=False)
print(U)
print(Vh)
np.sum(s>0.001)

[[-0.430  0.000 -0.790  0.436  0.000  0.000]
 [-0.416  0.000 -0.256 -0.873  0.000  0.000]
 [ 0.000 -0.667  0.000  0.000  0.596 -0.447]
 [ 0.000 -0.667  0.000  0.000 -0.745 -0.000]
 [-0.801  0.000  0.557  0.218  0.000  0.000]
 [ 0.000 -0.333  0.000  0.000  0.298  0.894]]
[[-0.265 -0.529  0.000  0.000 -0.138 -0.794]
 [-0.000 -0.000 -0.196 -0.981 -0.000  0.000]
 [ 0.037  0.074  0.000  0.000 -0.990  0.111]
 [-0.964  0.148  0.000  0.000 -0.000  0.222]
 [ 0.000 -0.395 -0.863  0.173 -0.000  0.263]
 [ 0.000 -0.732  0.465 -0.093  0.000  0.488]]


3

In [64]:
# QR base for comparison
B

array([[ 0.267, -0.000,  0.000],
       [ 0.535, -0.000,  0.000],
       [ 0.000,  0.196, -0.000],
       [ 0.000,  0.981,  0.000],
       [ 0.000,  0.000,  1.000],
       [ 0.802,  0.000, -0.000]])

In [65]:
B_svd = -Vh.T[:, 0:r]
B = B_svd
B

array([[ 0.265,  0.000, -0.037],
       [ 0.529,  0.000, -0.074],
       [-0.000,  0.196, -0.000],
       [-0.000,  0.981, -0.000],
       [ 0.138,  0.000,  0.990],
       [ 0.794, -0.000, -0.111]])